# The second part of the assignment, IDS 2021-2022
In this Jupyter notebook document all your results and the way you have obtained them. Please use the _Python environment_ provided for this part of the assignment. In addition to the _Jupyter notebook_, please submit _one zip-file_ containing  other outputs you have generated that are not included in this notebook (such as pdf, jpg, and others). Please make sure that the other outputs are easily identifiable, i.e. use names as requested in the corresponding question. _You do not need to include the datasets._

This is the _only_ submission that is required (Jupyter notebook + zip-file). A separate report is _not_ needed and will not be considered for grading. 

Give your commented Python code and answers in the corresponding provided cells. Make sure to answer all questions in a clear and explicit manner and discuss your outputs. _Please do not change the general structure of this notebook_. You can, however, add additional markdown or code cells if necessary. <b>Please DO NOT CLEAR THE OUTPUT of the notebook you are submitting! </b>

<font color="red"> Please make sure to include the names and matriculation numbers of all group members in the slot provided below. </font> If a name or a student id is missing, the student will not receive any points.

<font color="red">Plan your time wisely. </font> A few parts of this assignment might take some time to run. It might be necessary to consider time management when you plan your group work.

Hint: RWTHmoodle allows multiple submissions, with every new submission overwriting the previous one. <b>Partial submissions are therefore possible and encouraged. </b> This might be helpful in case of technical issues with RWTHMoodle, which may occur close to the deadline.


<font color="red"><b>Student Names and IDs:
    
    1. Daniel Weißen (427 492)
    
    2. Felix Meyer (378 959)
    
    3. Lars Kreuzberg (368 924)

# Setup

In [ ]:
### Display
from IPython.core.display import display, HTML

### Widgets
#import ipywidgets as widgets

### Data Handline
import pandas as pd
# avoid collapsed view for pandas dataframes
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

### Utility
import math
import string
import datetime
import pickle

### Plotting
# Matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
# Matplotlib toolkits
from mpl_toolkits.axes_grid1 import make_axes_locatable
# Seaborn
import seaborn as sns
# Plotly
import plotly.graph_objects as go
# Gespatial data with cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

### Sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


### Frequent Pattern Mining
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules as arule

### Text Mining
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# nltk.download('stopwords')
# nltk.download('wordnet')

### PM4Py
import pm4py
# Log Handling
from pm4py.objects.conversion.log import converter as log_converter
# Statistics
from pm4py.statistics.traces.generic.log import case_statistics
# Filtering
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.algo.filtering.log.attributes import attributes_filter
# Discovery and Conformance Checking
from pm4py.algo.evaluation.replay_fitness import algorithm as replay_fitness_evaluator
from pm4py.objects.conversion.process_tree import converter as pt_converter
# Visualization
from pm4py.visualization.process_tree import visualizer as pt_visualizer
from pm4py.visualization.petri_net import visualizer as pn_visualizer

pd.options.display.max_rows = 50

In [ ]:
# Pandas indexing
idx = pd.IndexSlice

Be careful if you use the **matplotlib widget** magic. If you do not close the created plots, previous plots may change if you create a new one.
So if you use this magic command, be careful about your outputs in your final submission.

In [ ]:
%matplotlib inline

# Q1 - Preprocessing (20 points)
In this question, we consider a US-census dataset (**census_data.csv**).
Each row contains statistics of a certain tract on a variety of, particularly income- and work-related, life aspects of US citizens.
Short column description:

| Column | Description |
| ------ | ----------- |
| CensusTract | Tract |
| State | State |
| County | County |
| TotalPop | Total population  |
| Men | Number of men |
| Women | Number of women |
| Hispanic, White, Black, Native, Asian, Pacific | Percentage of ethnic group |
| Citizen | Percentage of citizen |
| Income | Median household income |
| IncomePerCap | Income per capita |
| Poverty | Poverty rate |
| ChildPoverty | Child poverty rate |
| Professional | Employed in management, business, science, and arts (percentage) |
| Service, Office, Construction, Production | Other profession fields (percentage) |
| PrivateWork, PublicWork | Employed in private / public sector (percentage) |
| Drive, Carpool, Transit, Walk, OtherTransp | Means of commuting (percentage) |
| WorkAtHome | Working at home (percentage) |
| MeanCommute | Mean time for commuting |
| Employed | Number of employed |
| SelfEmployed | Self-employed (percentage) |
| FamilyWork | Unpaid family work (percentage) |
| Unemployment | Unemployment rate |


## Loading the Data and Initial Quality Investigation

**a)** Load the dataset into a dataframe `df`. <font color='red'>Use the CensusTract as index for your dataframe</font>. In doing so, ensure that the index is valid, that is, it does not contain any duplicate entries.

**In the subsequent questions, only modify the dataframe `df` if explicitly requested. However, you can always create working copies.**

In [ ]:
df = pd.read_csv('./dataset/census_data.csv', index_col='CensusTract')
# I don't know if it is necessary to explicitely check if no index values appear twice, if yes use:
# index = list(df.index)
# len(index) != len(set(index))

**b)** Show the data types of the dataframe columns as well as the first few rows. On the first sight, are there any data type problems (e.g., numerical columns having a non-numerical data type)? 

In [ ]:
df.dtypes

In [ ]:
df.head()

The column 'Employed' shows total number of people being employed whereas 'Unemployment' shows the percentage of unemployed people. Also the columns 'Men' and 'Women' are given as floating point numbers even though they are a total count and only integers make sense.

**c)** To improve performance and memory usage (in particular for large datasets) it is important to use **categorical** columns whenever suitable. 
Are there any categorical column candidates? Explain your answer. \
Afterwards, convert these columns into categorical columns.

In [ ]:
df = df.astype({"State":'category', "County":'category'})
# comment: I think county could actually be subcategories but this does not result in any memory improvements right?

Candidates for categorical columns are the Columns 'State' and 'County'.

**d)** To select a good strategy to deal with missing data, it is important to get an overview over the general data distribution.
Show the basic statistics for the dataset and create 6 boxplots for the following column groupings:

    ['TotalPop', 'Men', 'Women', 'Citizen', 'Employed'],
    ['Income', 'IncomePerCap'],
    ['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific'],
    ['PrivateWork', 'PublicWork', 'SelfEmployed', 'FamilyWork', 'Unemployment', 'Poverty', 'ChildPoverty'],
    ['Professional', 'Service', 'Office', 'Construction', 'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome'],
    ['MeanCommute']

Can you spot any (severe) data quality problems, in particular, are there unrealistic values (also considering the semantics)?

*Hint: Use the `df.describe()` function.*

In [ ]:
df.describe()

In [ ]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning) # to suppress a warning (maybe because i don't use provided env?)
df.boxplot(column=['TotalPop', 'Men', 'Women', 'Citizen', 'Employed'])

In [ ]:
df.boxplot(column=['Income', 'IncomePerCap'])

In [ ]:
df.boxplot(column=['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific'])

In [ ]:
df.boxplot(column=['PrivateWork', 'PublicWork', 'SelfEmployed', 'FamilyWork', 'Unemployment', 'Poverty', 'ChildPoverty'])

In [ ]:
df.boxplot(column=['Professional', 'Service', 'Office', 'Construction', 'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome'])

In [ ]:
df.boxplot(column=['MeanCommute'])

the minimum value is 0 -> there are censuses without any entries

## Handling Missing Values
(In the following task you can assume that every NAN entry in the dataframe is actually a missing value. This can paritally be justified by the fact that pandas did not have problems inferring the "proper" datatypes (e.g., numbers types as string would result in an object column) and your subsequent check of the data types. Therefore, you can use `df.isna()` as a proxy indicator for missing values.)

**e)** Simply filling missing entries is usually not a good idea. Therefore, you should first analyze the quantity of missing values and check for patterns of missing values.

To this end, compute the following statistics on missing values:
1) How many entries does the dataframe have? (To relate this to the number of entries missing)
2) How many missing values do we have?
3) How many rows have at least a single missing value?
4) Count the number of missing values per column.
5) Count the number of missing values per row and aggregate them - i.e., show the number of rows that suffer from x missing values.

What do you observe?

In [ ]:
num_total = len(df.columns)*len(df.index)
num_missing = df.isna().sum().sum()
print('Total number of entries: ', num_total)
print('Total number of missing entries: ', num_missing)
print('Percentage: ', num_missing/num_total*100)
print()
nan_sums_rows = df.isna().sum(axis=1)
num_rows_with_missing = len(df.index) - nan_sums_rows.value_counts()[0]
print('Number of rows with at least one missing value:', num_rows_with_missing)
print()
nan_sums_cols = df.isna().sum()
print('Missing values per column:')
print(nan_sums_cols)
print()
print('Missing values per row aggregated:')
print(nan_sums_rows.value_counts())

In total 22469 (~0.89%) of the values are missing distributed over 2008 rows. Most values are missing for columns 'Income' and 'ChildPoverty'. All values for the columns 'State', 'County', 'TotalPop', 'Citizen' and 'Employed' are available. Most rows have no missing values. If there are missing values it seems to be either very little (1-3) or many at once (20 or 27).

**f)** We decide to **remove all rows from `df` where the total population is zero**. \
Given the preceding results, how do you evaluate this strategy? Try to motivate your argumentation by additional short analysis results (see hint for an inspiration).

*Hint: It might be interesting to have a look at the rows with zero population. Afterwards, you can provide some analysis results that show that your (potential) observation generalizes to all rows with zero population.*

In [ ]:
# get rows with zero population
zero_pop = df.copy(deep=True)
zero_pop = zero_pop[zero_pop.TotalPop == 0]
print('There are {} rows with 0 population'.format(len(zero_pop)))
print('number of NaN values per column:')
zero_pop.isna().sum()

We observe that if the population is 0, most other values are missing. Therefore it makes sense to drop those rows.

In [ ]:
# drop rows were TotalPop is 0
df = df.loc[df.TotalPop != 0]

## Imputing Missing Values

**g)** The previous analysis showed that there are missing values in the 'Men' and 'Women' columns.\
How would you impute these values? \
Motivate your approach and apply it to `df`.

*Hint: Do not forget about the semantics of the columns.*

Idea: calculate the missing value for men from the value for total population and women and vice versa. We assume that each person is either a Man or a Woman.

In [ ]:
# check if there are columns where both 'Men' and 'Women' is missing
len(df[(df.Men.isna()) & (df.Women.isna())])

There are no columns were both values are missing so we can proceed as planned.

In [ ]:
df['Men'] = df['Men'].fillna(df.TotalPop - df.Women)
df['Women'] = df['Women'].fillna(df.TotalPop - df.Men)

In [ ]:
assert df[['Men', 'Women']].isna().sum().sum() == 0

**h)** Finally, impute the remaining missing values in `df` using the knn-imputation method.

1) Before you impute the remaining missing values, you should improve the data semantics consistency by turning the columns 
    
        ['Men', 'Women', 'Citizen', 'Employed']
    
    into percentage scores as well. To this end, divide these values by the total population (i.e., 'TotalPop')

In [ ]:
for key in ['Men', 'Women', 'Citizen', 'Employed']:
    df[key] = df[key] / df.TotalPop * 100

2) Impute the missing values using the knn-imputation method.
    To this end, apply the following steps:
        1) Create a working copy `df_tmp` of your dataframe.
        2) Drop the columns `['State', 'County']` from `df_tmp`. On the one hand, this makes the following steps easier because we only have to deal with numerical columns; on the other hand, an alternative one-hot encoding is also problematic as this will cause our feature dimensionality to explode!
        3) Normalize the data in `df_tmp` (e.g., Standard score normalization). If the features have very different scales, even though we are mostly using percentages, knn can become very biased.
        4) Impute the missing values considering five neighbors.
        5) Invert the transformation applied upfront to enable more meaningful and intuitive visualizations.
        6) Append the columns `['State', 'County']`
 
In the end, `df` should not contain missing values and have columns `['State', 'County']`.

*Hint: Be careful with the indices of your dataframes.*


In [ ]:
# create working copy
df_tmp = df.copy(deep=True)
# drop columns
df_tmp = df_tmp.drop(['State', 'County'], axis=1)

In [ ]:
# scale data with standard scaler
x = df_tmp.values # returns numpy array
standard_scaler = StandardScaler()
x_scaled = standard_scaler.fit_transform(x)

In [ ]:
# fill NaN values via knn-imputation
knn_imputer = KNNImputer(n_neighbors=5)
x_imputed = knn_imputer.fit_transform(x_scaled)

In [ ]:
# rescale data
x_rescaled = standard_scaler.inverse_transform(x_imputed)
df_tmp = pd.DataFrame(x_rescaled, index=df_tmp.index, columns=df_tmp.columns)

In [ ]:
df_tmp['State'] = df['State']
df_tmp['County'] = df['County']
df = df_tmp

In [ ]:
assert 'State' in df.columns
assert 'County' in df.columns
assert df.isna().sum().sum() == 0
assert df['Hispanic'].min() > -0.01
assert df['Hispanic'].max() < 101

## Data Integration

**i)** In the final preprocessing step, you should integrate one additional source of data into the preprocessed dataframe `df`. 
As the data has a natural geospatial dimension, you are going to endow each tract with its geographic coordinate.
To this end, load **coordinates.csv**. Integrate the two data sources exploiting the correspondence between 'CensusTract' and 'GEOID'.
Finally, drop the 'USPS' column.

In [ ]:
# load coordinates dataset
coords = pd.read_csv('./dataset/coordinates.csv', index_col='GEOID')
# join both datasets
df = df.join(coords)
# drop 'USPS'
df = df.drop(['USPS'], axis=1)
df

In [ ]:
assert 'Men' in df.columns
assert 'County' in df.columns
assert 'INTPTLONG' in df.columns
assert 'INTPTLAT' in df.columns

# Q2 - Visualization (15 points)
In this task, you will analyze the data that you preprocessed in question 1 (**census_data.csv**). In particular you will analyze income-related aspects, using different means of visualization.

Start with the following preprocessed and integrated dataframe `df`. \
Note that it has a similar structure to the dataframe that you should obtain from the previous task, however, the values have been modified!

**Library usage:** This notebook imports a couple visualization libraries that have a significat overlap in terms of functionalities. Therefore, you are free to use any of these libraries (and those in the environment in general) to implement the following questions as long as your resulting plot compilies with the explicitly mentioned requirements.

In [ ]:
df = pd.read_pickle('./dataset/df_vis.pkl')

**a)** Visualize two histograms for 'Income' and 'IncomePerCap' in a **single plot**. Compare the two distributions; what do you observe?

In [ ]:
df[['Income', 'IncomePerCap']].plot.hist(bins=20, alpha=0.5)

It is observable that the income is in general higher than the income per capita (PCI) and the PCI is more spread. This could be because counties with a high population have an overall high income but the income averaged over the population is not so high. 


*income is median household income, pci is totalincome/capita so mean income*

## Aggregation for Visualization Pruposes
**b)** As the data contains too many rows for per-row visualizations, you should aggregate the data further before creating more interesting visualizations.
The following function will do the job for your; however, why didn't we simply run `groupby(...).mean()` to get the results for the columns
specified in `l_col`?

In [ ]:
def my_aggregation(df):
    l_col = ['Men', 'Women', 'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'Citizen', 'Income', 'IncomePerCap', 
             'Poverty', 'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction', 'Production', 'Drive', 'Carpool', 
             'Transit', 'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork', 'SelfEmployed', 
             'FamilyWork', 'Unemployment', 'INTPTLAT', 'INTPTLONG']
    df_tmp = df.copy()
    df_tmp.loc[idx[:, l_col]] = df_tmp.loc[idx[:, l_col]].mul(df_tmp['TotalPop'], axis=0)
    df_tmp = df_tmp.groupby(['State', 'County'], observed=True).sum()
    df_tmp.loc[idx[:, l_col]] = df_tmp.loc[idx[:, l_col]].div(df_tmp['TotalPop'], axis=0)
    
    return df_tmp

# multiply by totalPop (compute absolute value)
# group by state county
# divide by totalpop

In [ ]:
df = my_aggregation(df)

When we group multiple regions with different populations, the mean over the statistics depend on the population of the regions so that taking the mean over the percentages is only true, when the population of the regions is the same. This assumption however doen't (always) hold.

*why would the same region have different number of totalPop?*

**c)** Next, you should create an overview over column correlations particularly considering high/medium/and low incomes.

1) Create a copy `df_plot` of the aggregated dataframe.

In [ ]:
df_plot = df.copy(deep=True)

2) Append a column 'IncomeClass' to `df_plot` containing the 'Income' categories based on the following inter-percentile ranges:
    - low' iff the income is less than the 33% income percentile
    - 'medium' if the income is between the 33% and 66% percentile, and 
    - 'high' iff the income is above the 66% percentile

In [ ]:
df_plot = df_plot.assign(IncomeClass=pd.qcut(df_plot['Income'], 
                                             [0, 0.33, 0.66, 1], 
                                             ['low', 'medium', 'high']))

3) Project the dataframe on the columns that contain percent values (for the sake of readability), that is:

        ['Men', 'Women', 'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'Poverty', 'ChildPoverty', 
        'Professional', 'Service', 'Office', 'Construction', 'Production',
        'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome',
        'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork', 'SelfEmployed',
        'FamilyWork', 'Unemployment', 'IncomeClass']
        

In [ ]:
df_plot = df_plot[['Men', 'Women', 'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'Poverty', 'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction', 'Production','Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome','MeanCommute', 'Employed', 'PrivateWork', 'PublicWork', 'SelfEmployed','FamilyWork', 'Unemployment', 'IncomeClass']]

4) Create a parallel coordinates diagram that uses the 'IncomeClass' for coloring the lines. Rotate the x-axis labels by 90° to make them easier to read.
Briefly discuss your results. Do you observe any correlations? Please explain.
    

In [ ]:
mpl.pyplot.figure(figsize=(25,12), dpi=120)
plot = pd.plotting.parallel_coordinates(df_plot, 'IncomeClass', alpha=0.3, color=["green", "violet", "blue"])
plot.tick_params(axis='x', labelrotation=90)
plt.show()
# comment: rotate label

Most observations aren't interesting and obvious like the negative correlation between men and women. Also the correlations in groups of the same category like "drive" and "Carpool" are obvious because if many people drive, they don't make use of carpools, which implies a negative correlation in this case.
Meaningful Obervations: 
- positive correlation between the amount of production in an area and people driving
- negative correlation between Poverty and Professional Workers, same for child poverty
- positive correlation between Poverty and service workers
- positive correlation between poverty and amount of black people
- negative correlation between unemployment and amount of white people + asian people


## Advanced Visualization: Map

**d)** In this task, you are going to create an advanced visualization that exploits the geospatial nature of the data, that is, you will project the average 'Income' of each county and its population onto a map of the USA. \
You can use the following code to create a suitable map extend.

    ax.set_extent([-125, -66.5, 20, 50], ...)
        
Given this instance, plot one marker (e.g., circular marker) for each row in our aggregated dataset onto the map.
The color encoding should show the average 'Income' of the corresponding county, while the size should be chosen according to its population ('TotalPop'). If you want to modify the dataframe, create a **working copy** beforehand.

What do you observe?


In [ ]:
df.head()

# Create projection
proj = ccrs.PlateCarree()

# Colormap
cmap = mpl.cm.get_cmap('RdYlGn')
# Normalization
norm = mpl.colors.Normalize(vmin=0, vmax=df['Income'].max())
# Get colors according to color map specification above
np_col = cmap(norm(df['Income']))

# Compute the circle size
# To show you 
size_scaler = MinMaxScaler()
np_size = 600 * size_scaler.fit_transform(df[['TotalPop']])

# Create figure
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection=proj)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.STATES)
ax.set_extent([-125, -66.5, 20, 50])
ax.scatter(df['INTPTLONG'], df['INTPTLAT'], color=np_col, s=np_size, marker='o')

# Add colorbar to the plot
divider = make_axes_locatable(ax)
cax = divider.new_horizontal(size='5%', pad=0.2, axes_class=plt.Axes)
fig.add_axes(cax)
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=cax, orientation='vertical')
cbar.set_label('Median Household Income', rotation=270)
# Figure title
fig.suptitle('Median Household Income in Counties of the US')
fig.tight_layout()

plt.show()

We can observe that in most larger cities, the income is high. In the inner country, with cities with lower populations, the income is in general lower. An exception is the state of Florida, where the overall income seems to be low, even though there are some big cities like Orlando and Miami. We can also see that more people live in the East while especially northern states have low populations.

*arent there quite some big green points in FLorida?*

# Q3 - Frequent Itemsets and Association Rules (12 points)


## Frequent Itemsets

**a)** Carry out some preprocessing steps before starting the analysis:

1) Load the `customer_data.csv`.

2) Select 90% of the `customer_data` dataset by random sampling. Use the matriculation number of one of the group members as seed.

3) After completing this preprocessing step, export your final dataset as `customer_data_2.csv` and use it for the next steps of the assignment.

In [ ]:
customer_data = pd.read_csv('./dataset/customer_data.csv')
customer_data_sample = customer_data.sample(frac=0.9, random_state=427492)
customer_data_sample.to_csv('./dataset/customer_data_2.csv')
customer_data_sample.head()

**b)** In this part, we want to get to know our customers by looking at the typical shared characteristics (e.g. "Married customers in their 40s like wine"). This would correspond to the itemset {Married, 40s, Wine}. 

1) Create a new dataframe called `customer_data_onehot` such that rows correspond to customers (as in the original data set) and columns correspond to the categories of each of the ten categorical attributes in the data. The new dataframe should only contain boolean values (True/False or 0/1s) such that the value in row $i$ and column $j$ is True (or 1) if and only if the attribute value corresponding to the column $j$ holds for the customer corresponding to row $i$. Display the dataframe.

*Hint: For example, for the attribute "Education" there are 5 possible categories: 'Graduation', 'PhD', 'Master', 'Basic', '2n Cycle'. Therefore, the new dataframe must contain one column for each of those attribute values.* 

In [ ]:
customer_data_sample = pd.read_csv('./dataset/customer_data_2.csv')
customer_data_onehot = pd.get_dummies(customer_data_sample)
customer_data_onehot = customer_data_onehot.rename(columns={'Unnamed: 0': 'ID'}).set_index("ID")
customer_data_onehot.head()

2) Use the apriori algorithm to find the frequent itemsets with **min_support = 0.3** from the `customer_data_onehot` dataframe. Show the frequent itemsets that contain at least **3** items.

*Hint: The apriori algorithm of mlxtend needs a dataframe containing only boolean values as input.*

In [ ]:
frequent_itemsets = apriori(customer_data_onehot, min_support = 0.3, use_colnames = True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x)) 
frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 3]
display(frequent_itemsets)

**c)** In the following we will investigate the effect of using the apriori property when determining the candidates for the frequent itemsets.

1) Implement the following join- and prune steps of the Apriori algorithm: \
   **join function:** a function that, given the frequent itemsets of size k, generates and yields a list of itemsets of size k+1. Only itemsets that share exactly k elements should be merged. \
   **prune function:** Given the set of candidate itemsets of size k+1 and the set of frequent itemsets of size k, remove the candidate sets that contain an infrequent subset of size k and return the rest.

In [ ]:
def join(itemsets_k):
    k = len(itemsets_k[0])
    extended_sets = []

    for outer_set in itemsets_k:
        for inner_set in itemsets_k:
            if len(outer_set.intersection(inner_set)) == k-1:
                extended_sets.append(set(outer_set).union(set(inner_set)))

    return extended_sets

In [ ]:
def prune(itemsets_k, itemsets_k1):
    k = len(itemsets_k[0])
    freq_itemsets = []

    for freq_set in itemsets_k1:
        is_frequent = True

        for item in freq_set:
            set_copy = freq_set.copy()
            set_copy.remove(item)
            if set_copy not in itemsets_k:
                is_frequent = False
                break
            
        if is_frequent:
            freq_itemsets.append(freq_set)

    return freq_itemsets

2) To see the effect of the apriori property, compare the number of candidate itemsets of size 4 obtained with and without pruning from the itemsets of size three for different values for min_support. To this end, generate a list of tuples *(min_sup, C4_size, C4_size_pruned, L4_size)* as follows:

For $\textrm{min_support} \in [0.1,0.2,...,0.8,0.9,1]$, repeat:

1. Obtain all frequent itemsets of size three using the apriori algorithm.

2. Using the result from 1., generate all itemsets of size four by applying your **join function** $\rightarrow$ C4_size.

3. Prune the result from 2. using your **prune function** $\rightarrow$ C4_size_pruned.

4. Compute the frequent itemsets of size four by using the apriori algorithm $\rightarrow$ L4_size.

In [ ]:
support_vals=[0.02, 0.04, 0.08, 0.16, 0.32]
plot_data = []

for support in support_vals:
    frequ_itemsets = apriori(customer_data_onehot, use_colnames = True, min_support=support)
    frequ_itemsets['length'] = frequ_itemsets['itemsets'].apply(lambda x: len(x)) 

    frequ_itemsets_size3 = (frequ_itemsets[frequ_itemsets['length'] == 3])['itemsets'].tolist()
    frequ_itemsets_size4 = (frequ_itemsets[frequ_itemsets['length'] == 4])['itemsets'].tolist()

    if frequ_itemsets_size3:
        itemsets_size4_dirt = join(frequ_itemsets_size3)
        itemsets_size4_clean = prune(frequ_itemsets_size3, itemsets_size4_dirt)
        
        plot_data.append([support, len(itemsets_size4_dirt), len(itemsets_size4_clean), len(frequ_itemsets_size4)])

df_plot = pd.DataFrame(plot_data, columns=['support', 'before prune', 'after prune', 'apriori'])
display(df_plot)

3) Plot the number of candidate sets with and without pruning and the number of frequent itemsets of size four against the corresponding min_sup value. Interpret the plot.

In [ ]:
ax = plt.gca()

for col in ['before prune', 'after prune', 'apriori']:
    df_plot.plot(kind='line',x='support',y=col, ax=ax)

plt.show()

We can obverse that pruning can reduce the number of itemsets by a significant amount. However the actual target what we get with the apriori algorithm is still a lot lower, meaning that the procedure is not sufficient to make predictions on the actual number of targets. All lines converge to zero which makes sense since the more items we take into account, the less fullfill the condition of being contained in a set after the apriori algorithm.

d) Use the FP-Growth algorithm to obtain all frequent itemsets with **min_support = 0.3** from `customer_data_onehot`.

In [ ]:
frequent_itemsets = fpgrowth(customer_data_onehot, min_support=0.3, use_colnames = True)
display(frequent_itemsets)

## Association Rules

**d)** In the following, you should generate association rules from the frequent itemsets.

1) Using only the frequent itemsets with min_support=0.3, generate different association rules using minimum confidence equal to 0.6 as a metric. Show the association rules.

In [ ]:
assoc_rules = arule(frequent_itemsets, metric = 'confidence', min_threshold = 0.6)
display(assoc_rules)

2) From the association rules obtained in task (d) 1), provide the three rules with the highest lift. Comment on them.

In [ ]:
(assoc_rules.sort_values('lift', ascending=False))[0:3]

Since the lift for all 3 rules is over 1 or even over 2, it means that there is a positive correlation between antecedents and consequents. So the occurence of the antecedents implies that the consequents are also likely to occur. The relatively high confidence of over 78% for all three values confirm that.

# Q4 - Text Mining (15 points)
In this question, you will use the scripts of some Harry Potter movies. First, you will try to predict the character given a line in the script. Afterwards, using N-grams, you will generate sentences for some of the characters.

**a)** In this part, you will preprocess and reconstruct the data to make it suitable for the following tasks.

1) Load each of the datasets <b>hp_1.csv</b>, <b>hp_2.csv</b>, and <b>hp_3.csv</b> into its own dataframe and show the set of characters (here: a fictional character) appearing in each dataset.

In [ ]:
# load datasets
hp_1 = pd.read_csv('./dataset/hp_1.csv')
hp_2 = pd.read_csv('./dataset/hp_2.csv')
hp_3 = pd.read_csv('./dataset/hp_3.csv')

# strip leading and trailing whitespaces and convert to lower case 
hp_1 = hp_1.applymap(lambda x : x.strip().lower())
hp_2 = hp_2.applymap(lambda x : x.strip().lower())
hp_3 = hp_3.applymap(lambda x : x.strip().lower())

# compute characters that appear in each of the datasets
from functools import reduce
print(reduce(np.intersect1d, (hp_1['Character'], hp_2['Character'], hp_3['Character'])))

2) Merge the three datasets into a single dataframe called `hp_df` that comprises only the lines spoken from one of the four characters *Harry, Hermione, Dumbledore, and Snape*. Your new dataframe must contain two columns: one for the (four) characters and the other for the lines. You can name those columns "Character" and "Sentence" as in the original data. \
    Make sure that `hp_df` contains a single unique spelling for each of the characters. \
    Make sure that `hp_df` includes all lines (here: script lines) of a character even if this character is spelled slightly differently (e.g., Dumbledore or dumbledore) in the original dataset. \
    Show the first few lines of your dataframe.
    
*Hint: Be aware of white space characters!*

In [ ]:
hp_df = pd.concat([hp_1, hp_2, hp_3])
hp_df = hp_df.loc[hp_df['Character'].isin(['harry', 'hermione', 'dumbledore', 'snape'])]
hp_df.head()

Note: We neglect the typo in 'hermoine' and examples with multiple people. The number of occurences of these are very low so this should be okay.

3) Create the `hp_sampled` dataset which includes 90% of the `hp_df` data. Use the matriculation number of one of the group members as seed. Export the sampled dataset. 

In [ ]:
hp_sampled = hp_df.sample(frac=0.9, random_state=427492)
hp_sampled.to_csv('./dataset/hp_sampled.csv')

**b)** In this part, you are going to train a classifier that, given a line from the script, predicts the character. For each character, the data contains many sentences belonging to that character. Note that sometimes the "Sentence" column in the original dataset contains more than one sentence. The set of sentences for each character should be seen as the set of example documents belonging to that character (the class). Each individual sentence is a single document. The whole corpus consists of all the individual sentences.

1) Create a new dataframe called `hp_processed` from the dataframe `hp_sampled` such that the new dataframe contains again the columns "Character" and "Sentence", but every entry in the "Sentence" column must be a single sentence. Display the shape of the dataframe and compare it to the shape of `hp_sampled`.

In [ ]:
# rename index so no id appears multiple times
hp_processed = hp_sampled.set_axis(range(len(hp_sampled)))

s = hp_processed.Sentence.map(sent_tokenize).apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1) # to line up with df's index
s.name = 'Sentence' # needs a name to join
del hp_processed['Sentence']
hp_processed = hp_processed.join(s)

2) Split the preprocessed data `hp_processed` into training (80%) and test (20%) data preserving the distribution based on "Character". 

In [ ]:
train, test = train_test_split(hp_processed, test_size=0.2, random_state=427492, stratify=hp_processed.Character)

3) Preprocess the training and test corpus (to lowercase, no punctuation, tokenization, lemmatization, and stopword removal) and obtain a boolean document-term matrix (i.e, each row in the matrix contains only 1s and 0s depending on whether a particular word appears in a sentence or not). Train a logistic classifier on the training corpus with the character as target feature. Use the classifier to predict the character of the sentences in the test corpus and show its accuracy on the test corpus. Comment on the result.

In [ ]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
stop_list = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def preprocess(sentences):
    corpus = []

    for sentence in sentences:
        tokenized = tokenizer.tokenize(sentence)
        filtered = [word.lower() for word in tokenized if word.lower() not in stop_list]
        lemmatized = [lemmatizer.lemmatize(word) for word in filtered]
        corpus.append(lemmatized)
    
    return corpus

In [ ]:
corpus_train = preprocess(train.Sentence)
corpus_test = preprocess(test.Sentence)

count_vect = CountVectorizer()
count_vect.fit(pd.Series([' '.join(s) for s in corpus_train]))

train_count_matrix = count_vect.transform(pd.Series([' '.join(s) for s in corpus_train]))
test_count_matrix = count_vect.transform(pd.Series([' '.join(s) for s in corpus_test]))

In [ ]:
count_vect.vocabulary_

In [ ]:
print('Creating classifier')

classifier = LogisticRegression()

train_vectors = train_count_matrix
train_target = train.Character

test_vectors = test_count_matrix
test_target = test.Character

classifier = classifier.fit(X=train_vectors, y=train_target)

print('Evaluating model on test set')

print('Classifier accuracy on test set:')
print(classifier.score(test_vectors, test_target))

In [ ]:
from sklearn.metrics import confusion_matrix
predictions = classifier_doc2vec.predict(test_vectors)
confusion_matrix(test.Character, predictions, labels=['harry', 'hermione', 'dumbledore', 'snape'])

The performance of the logistic classifier is pretty poor with 56% accuracy. Especially considering, that the classifier learns that the character Harry has more occurences in the data. This is probably due to the sparseness of the data. It is very unlikely that similar sentences appear multiple times, especially since similar words cannot be determined from the count matrix representation. Also most sentences have barely any words because of the stopwaord removal.

4) Next, you are going to perform the same predicting task based on doc2vec.

1. Preprocess the training corpus (to lowercase, no punctuation, tokenization, lemmatization, and stopword removal). 
2. Create a doc2vec model to reduce the dimension of the document vector. Choose a vector size 4-8 and ignore all words whose count is lower than 3.
3. Train the doc2vec model on the training data (thus creating an embedding).
4. Use the created embedding to convert the training set to a set of document vectors.
5. Train a logistic classifier on the train data with the character as target feature.
6. Show the accuracy of prediction on the test data and comment on it.

In [ ]:
# create TaggedDocument
tagged_docs_train = [TaggedDocument(doc, [i]) for i, doc in enumerate(corpus_train)]

# determining parameters of the model   
model = Doc2Vec(vector_size=8, min_count=3)

# building the vocabulary    
model.build_vocab(tagged_docs_train)

training_examples = model.corpus_count

# document embedding, train the model on the training examples (docs)
model.train(corpus_iterable=tagged_docs_train, total_examples=training_examples, epochs=100)

In [ ]:
print('Creating classifier')

classifier_doc2vec = LogisticRegression()

train_vectors = [model.infer_vector(doc) for doc in corpus_train]
train_target = train['Character']

test_vectors = [model.infer_vector(doc) for doc in corpus_test]
test_target = test['Character']

classifier_doc2vec = classifier_doc2vec.fit(X=train_vectors, y=train_target)

print('Evaluating model on test set')

print('doc2vec - Classifier accuracy on test set:')
print(classifier_doc2vec.score(test_vectors, test_target))

**c)** For the following tasks use the `hp_processed` (the data before splitting into training and test data).

1) For each character, create a list containing all sentences of that character.
    For each character separately, build a bigram language model using MLE. Do not perform stemming and stopword removal for this task, but apply other preprocessing steps such as to lowercase, no punctuation, and tokenization. Use both right and left padding.

In [ ]:
def preprocess_simple(sentences):
    corpus = []
    for sentence in sentences:
        tokenized = tokenizer.tokenize(sentence)
        filtered = [word.lower() for word in tokenized]
        corpus.append(filtered)

    return corpus

In [ ]:
characters = ['harry', 'hermione', 'dumbledore', 'snape']

hp_data = [hp_processed[hp_processed.Character == c] for c in characters]
hp_corpora = [preprocess_simple(d.Sentence) for d in hp_data]

In [ ]:
def build_lm(n,corpus):
    # the padded_everygram_pipeline goes through the corpus, applies left and right padding to the sentences (adding <s> and </s>)
    # and obtains the tuples of a given order together with the vocabulary
    padded_tuples, vocab = padded_everygram_pipeline(n, corpus)
    # generate (an empty) ngram language model for some n>0
    lm = MLE(n)
    # generate probabilities (model) given the list of n-grams and the vocabulary
    lm.fit(padded_tuples, vocab)
    
    return lm

In [ ]:
bigrams = [build_lm(2, corpus) for corpus in hp_corpora]

2) For each character, use the created language model to generate a sentence of ten words. Display the sentences.

In [ ]:
for i in range(len(characters)):
    print('{}:\t {}'.format(characters[i], ' '.join(bigrams[i].generate(10))))

3) Build a 4-gram model with the same data as in the previous task. Use both right and left padding.

In [ ]:
fourgrams = [build_lm(4, corpus) for corpus in hp_corpora]

4) For each character, use the created 4-gram language model to generate a sentence of ten words. Display the sentences.

In [ ]:
for i in range(len(characters)):
    print('{}:\t {}'.format(characters[i], ' '.join(fourgrams[i].generate(10))))

5) Compare the sentences generated by the bigram model with the sentences generated by the 4-gram model.

*Your markdown for your discussion.*

# Q5 - Process Mining (23 points)

In this task, we consider a simulated process of students that participate in an online course.
The course comprises 6 batches of lecture material as well as a mandatory assignment to be delivered in two parts. (Note that in this process, it is not required to achieve a certain score in the assignment in order to be admitted to the exam.)

While there are strict deadlines for the assignment and the exam, there is only a recommended schedule for the lecture material (i.e., consume material in order).

The system logs for every student, among other activities, when he downloads a certain lecture material batch. 

## Loading the Data and Basic Statistics

**a)** Load the data **log.csv** and create a PM4Py event log. In doing so, use the following column mapping:
 - 'Activity' is the activity key
 - 'Student' is the case ID
 - 'Timestamp' is the timestamp

In [ ]:
# Your code
log = pd.read_csv('dataset/log.csv')
df = pm4py.format_dataframe(log, case_id='Student', activity_key='Activity', timestamp_key='Timestamp')
log = log_converter.apply(df)
display(df)

**b)** Compute the following basic information:
- Number of events
- Number of cases
- Earliest timestamp
- Latest timestamp
- Number of trace variants

In [ ]:
# Your code
print('number of events: {}'.format(df.shape[0]))
print('number of cases: {}'.format(len(log)))
print('earliest timestamp: {}'.format(df.sort_values('time:timestamp')['time:timestamp'].iloc[0]))
print('latest timestamp: {}'.format((df.sort_values('time:timestamp', ascending=False))['time:timestamp'].iloc[0]))
print('number of trace variants: {}'.format(len(variants_filter.get_variants(log))))

**c)** Usually, it is insightful to have a look at the distribution of the variants in terms of how often a certain variant is present in the log.
Therefore, create a **scatter plot** that shows the distribution of the variants as follows:
- x-axis: The variants (in ascending order of their support)
- y-axis: Frequency of the variant in the log (total or relative)

In [ ]:
# Your code
variants_count = case_statistics.get_variant_statistics(log)
vc_df = (pd.DataFrame.from_dict(variants_count)).sort_values('count', ascending=True)
var_freq = [(len(vc_df)-index, row['count']) for index, row in vc_df.iterrows()]

plt.figure(figsize=(10,10))
plt.scatter(*zip(*var_freq))
plt.ylabel('frequency')
plt.xlabel('variant')
plt.yscale('log')
plt.show()
# comment: change type to scatter plot, but then the labels are unreadable

**d)** While the variant distribution shows potential standard process executions in terms of the activity ordering, the distribution of the case durations shows the typical timeframe of cases.
Create a histogram plot over the case durations. For the sake of readability, make sure that the x-axis labels (in this case the case durations) have an easily readable format, that is, your x-axis labels should look like this:
<br></br>
<div>
<img src="templates/caseDurationXAxis.png" width="500"/>
</div>

In [ ]:
# your code
td = datetime.timedelta(seconds=13700502.0)
print(td)

case_dur = pd.DataFrame.from_dict(case_statistics.get_cases_description(log))
case_dur_t = case_dur.transpose()

fig, ax = plt.subplots()
chart = plt.hist(case_dur_t['caseDuration'])

def timeTicks(x, pos):                                                                                                                                                                                                                                                         
    d = datetime.timedelta(seconds=x)
    return str(d)
formatter = mpl.ticker.FuncFormatter(timeTicks)
plt.xticks(rotation=90)
ax.xaxis.set_major_formatter(formatter)  

#, ax=ax, x="datetime",y="Duration"

## Discovery and Conformance Checking
Next, you are going to discover process models for different perspectives on the process. Moreover, you will evaluate how well the process models can represent the behavior present in the log (i.e., the fitness of the models).

### Filtering

**e)** Before discovering models, create three addtional perspectives onto the process by creating three additional event log from the log loaded in a):

1. Log containing only 30% of the most frequent traces (**log_varaint03**)
2. Log containing only 50% of the most frequent traces (**log_varaint05**)
3. Log containing only students that take the exam, that is, cases that end with 'Exam' (**log_exam**)

In [ ]:
# your code
log_varaint03 = variants_filter.filter_log_variants_percentage(log, percentage=0.3)
log_varaint05 = variants_filter.filter_log_variants_percentage(log, percentage=0.5)
from pm4py.algo.filtering.log.end_activities import end_activities_filter
log_exam = end_activities_filter.apply(log, ["Exam"])

### Inductive Miner and Replay Fitness

**f)** To get a better understanding of the processes in our four event logs (base log + three additional logs), create processes models using the **Inductive Miner**.
Concretely, for each of the 4 event logs, create two process models using the Inductive Miner with noise threshold **0 and 0.2**. Moreover, to access how well the model presents the logged behavior, apply conformance checking in terms of token-based replay to the model and the log from which it has been mined. Visualize each model as a process tree and as the corresponding Petri net.

In total, your cells should output 8 conformance scores, 8 process trees, and 8 Petri nets (for each log + noise threshold combination). Make sure that it is clear which model and conformance score belongs to log and parameter configuration.

For example, your output can look like this
#### Log: Base
##### IM threshold 0
>Fitness score

>Picture of the process tree

>Picture of the Petri net

##### IM threshold 0.2
>Fitness score

>Picture of the process tree

>Picture of the Petri net

**Describe your results**. How well do the models fit and, in particular, how do the models for
- log_variant05 and log (base log)
- log_variant05 and log_variant03
- log base and log_exam

differ in terms of the behavior that they allow?


In [ ]:
# your code
for item in [(log, 'full log'), (log_varaint03, '30% most frequent'), (log_varaint05, '50% most frequent'), (log_exam, 'ends with exam')]:
    print('log: ' + item[1])
    proc_tree = pm4py.discover_process_tree_inductive(item[0], noise_threshold=0)
    gviz = pt_visualizer.apply(proc_tree, parameters={pt_visualizer.Variants.WO_DECORATION.value.Parameters.FORMAT: "png"})
    pt_visualizer.view(gviz)

    net, im, fm = pt_converter.apply(proc_tree)
    pm4py.view_petri_net(net, im, fm, format='png')
    fitness = replay_fitness_evaluator.apply(item[0], net, im, fm, variant=replay_fitness_evaluator.Variants.TOKEN_BASED)
    display(fitness)

Observations:
- 30% most frequent visit almost all lectures and take the assignments and the exam
- 50% most frequent: more students skip events
- all logs: Looks similar to the 50% most frequent logs
- exam at the end: also looks similar to the 50% most frequent logs but there are a few less skip connections
The fitness for all the logs is the same so this value isn't interesting for observing differences

### Paths of Excellence

**g)** As lectures are in a constant urge to improve their courses in a way that participants learn as much as possible ;), you are facing the research question to identify *paths of excellence*.
In particular, you should identify how the studying behavior differs between excellent students (**final exam score greater than or equal to 85**) and non-excellent students (**final exam score less than 85**).
Try to answer this research questions using techniques from the preceding Process Mining questions. 

*Hint: There is no single unique solution (e.g., in terms of parameter choice); therefore, it suffices if your "design choices" are reasonble.* \
*Hint: The final exam score is point score that is associated with the "Exam" event.*


In [ ]:
# your code
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.objects.petri_net.obj import PetriNet, Marking



log_excellence = attributes_filter.apply_numeric(log_exam, 85, 100,
                                             parameters={attributes_filter.Parameters.ATTRIBUTE_KEY: "Points",
                                                         attributes_filter.Parameters.STREAM_FILTER_KEY1: "concept:name",
                                                         attributes_filter.Parameters.STREAM_FILTER_VALUE1: "Exam"})

log_non_excellence = attributes_filter.apply_numeric(log_exam, 0, 84,
                                             parameters={attributes_filter.Parameters.ATTRIBUTE_KEY: "Points",
                                                         attributes_filter.Parameters.STREAM_FILTER_KEY1: "concept:name",
                                                         attributes_filter.Parameters.STREAM_FILTER_VALUE1: "Exam"})

for item in [('log_excellence', log_excellence), ('log_non_excellence', log_non_excellence)]:
    print('log: ' + item[0])
    proc_tree = pm4py.discover_process_tree_inductive(item[1], noise_threshold=0)
    net, im, fm = pt_converter.apply(proc_tree)
    parameters = {pn_visualizer.Variants.FREQUENCY.value.Parameters.FORMAT: "png"}
    gviz = pn_visualizer.apply(net, im, fm, parameters=parameters, variant=pn_visualizer.Variants.FREQUENCY, log=item[1])
    dfg_visualization.view(gviz)

print(len(log_non_excellence))

It is clearly evident that the students who take an excellent exam attend the lectures in their intended order and complete the assignments. Likewise, each of these students attended the final Q&A. 
For the rest of the students, with a worse performance, one sees significantly more different connections between the courses, suggesting that courses are also skipped. For example, there are 28 students who started with Lecture 3 and almost 1000, and thus the majority, who started with Assignment 1 without having looked at any lectures before that. However, it is still evident that a majority of the students who have registered for the exam take all the events. So it seems that the order in which one attends the events has an influence on the result or when one starts to study, because the events are activated gradually.

## Performance and Frequency Decoration

**h)** While the discovery of a process model is the most prototypical step in a process mining analysis, its enrichment by frequency and performance statistics is a very common step too. To this end, enrich the Petri net that you discovered for log_exam using Inductive Miner with noise threshold 0.2 by frequency and performance information. Plot two Petri nets decorated with frequency and performance information, respectively.

Describe your results. Can you observe any problems (in particular with respect to the initial process description)?

In [ ]:
# your code
proc_tree = pm4py.discover_process_tree_inductive(log_exam, noise_threshold=0.2)
net, im, fm = pt_converter.apply(proc_tree)
parameters = {pn_visualizer.Variants.FREQUENCY.value.Parameters.FORMAT: "png"}
gviz = pn_visualizer.apply(net, im, fm, parameters=parameters, variant=pn_visualizer.Variants.FREQUENCY, log=log_exam)
dfg_visualization.view(gviz)
gviz = pn_visualizer.apply(net, im, fm, parameters=parameters, variant=pn_visualizer.Variants.PERFORMANCE, log=log_exam)
dfg_visualization.view(gviz)

Don't understand, why in these petri nets, there is no real order, in which the differnet events are taken, even though there are obviously overlaps.
# comment: formulate answer

## Process Mining Meets Advanced Visualization

### Studying Activity Heatmap

**a)** In this task, we are going to use advanced visualization techniques to create an overview over the course acitivities over time (**log.csv**).
In particular, you shall create a heatmap that shows how often activities occur in a particular week. 
Your heatmap should adhere to the following specification:
- y-axis: Shows the activity labels
- x-axis: Time in terms of course weeks. See the following example snippet:
<div>
<img src="templates/PMAV_HeatmapXAxisSnippet.png" width="100"/>
</div>
- data: The bucket counts should be derived from **log.csv**

Using this configuration, the bucket 2021-11-24 till 2021-12-01 with y-axis label "Exam" and value v would be read as:
In the week between 2021-11-24 and 2021-12-01 v exams took place.

Describe your result. Which patterns do you observe?

In [ ]:
# your code
from datetime import datetime, timedelta

log_grouped_week = df.copy(deep=True)
log_grouped_week['time:timestamp'] = log_grouped_week['time:timestamp'].apply(lambda x: (x - timedelta(days=x.weekday())))
log_grouped_week = log_grouped_week.astype({'time:timestamp': 'str'})
log_grouped_week['time:timestamp'] = log_grouped_week['time:timestamp'].apply(lambda x: x[0:10])

df_m = log_grouped_week.groupby(["time:timestamp", "concept:name"]).size().unstack(level=0)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(df_m, ax=ax)
# ax.set_xticklabels(log_grouped_week['Timestamp'].dt.strftime('%d-%m-%Y'))

Observed patterns:
- Registration for the exam occurred over a relatively long period of time probably up to the end of the registration period during the week of 2021/11/29.
- The submission phase of Assignment 1 and 2 was most likely in a 2-week time period
- The 6 lectures were first consumed in a relatively evenly spaced period from the beginning of the semester to the end of the semester with a cut at the first Assignment. Maybe due to the preperation for the assignment, people only consumend the lectures relevant for the first assignment until it needed to be submitted. However, it is not possible to say whether individuals also consumed them consecutively, as there is still a large overlap between lectures.
- 2 weeks before the exam and after the first assignment the lectures were repeated again by a small part of the students.

**b)** Can you relate the patterns that you observe in the heatmap to the process models that you discovered in question **Q5 - f)**?

*Hint: In contrast to the other questions, this question is deliberately less explicit. You may approach it having the following question in mind: \
Is there a pattern in the heatmap that explains why a certain process model shows a certain behavior/control flow?*

The heat map only shows which events took place at which point in time. However, in the case of overlaps, which can often be seen here, no statement can be made about what exactly the individual traces look like. Only an upper trend can be seen, which can be used to make assumptions about what the control flow will most likely look like. However, really explicit statements can only be made about how often individual events have taken place and at what time.

### Process Flow

**c)** Assume that another process analyst also had access to the event log. 
Given the data, he created the following novel entities:
- 'Block 1 Complete': The participant downloaded the entire material of the first lecture block (materials 1, 2, and 3). (Not considering when he downloaded it)
- 'Block 1 Incomplete': The participant did not download the entire material of the first lecture block
- 'Block 2 Complete': See 'Block 1 Complete'
- 'Block 2 Incomplete': See 'Block 1 Incomplete'
- 'Ass 1 Excellent': Participant scored at least 85 points in the first part of the assignment
- 'Ass 1 Not Excellent': Participant scored less than 85 points in the part of the assignment
- 'Ass 2 Not Excellent', 'Ass 2 Excellent', 'Exam Not Excellent', 'Exam Excellent': Similar to 'Ass 1 Excellent' and 'Ass 1 Not Excellent'
- 'Withdraw': Participant dropped the course

Based on the entities he derived a set of flow, for example, the flow between 'Block 1 Complete' and 'Ass 1 Excellent' describes how often a partipant who consumed the first lecture block scored excellent in the first part of the assignment.

The next cell loads the entities and flows for you. The flows are stored as a dictionary following the pattern:

    (source, target): flow_value
    
where source and target are indices into the entities list.

Create a **Sankey diagram** that visulizes these flows. Please use `plotly.graph_object.Sankey` to create the diagram.

Briefly **describe** your results.
Moreover, **discuss** this visualization considering your knowlege from the Process Mining task.

*Hint: Having a look at the Sankey diagram will make the analyst's idea behind the entities much clearer.*

In [ ]:
with open('./dataset/sankeyEntities.pkl', 'rb') as f:
    entities = pickle.load(f)
with open('./dataset/sankeyFlows.pkl', 'rb') as f:
    flows = pickle.load(f)

In [ ]:
# your code
l_source = []
l_target = []
l_values = []
for key in flows:
    l_source.append(key[0])
    l_target.append(key[1])
    l_values.append(flows[key])

# Define Sankes diagram
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = entities,
      color = "blue"
    ),
    link = dict(
      source = l_source,
      target = l_target,
      value = l_values
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

As analyzed before, it can be clearly seen that the students who completed both lecture blocks and passed the assignments excellently, also passed the exam with excellent results for the most part. In addition, a large part of the students who withdraw from the course, did so after the first block or after a not so successful first assignment. The number of drop-outs decreases the later the course is. Except after the second block, every student continued to the second assignment.

# Question 6 - Big Data (15 points) 

### Problem Description
You are working at a finance company that makes loans to individuals and businesses. As a process analyst in *business intelligence team*, you are expected to deliver data-driven insights to improve business processes of the company. Recently, your boss asked you to discover a comprehensive process model of 10 international branches using your big data skills. Your colleague already tried it using commercial on-premise tools, but, due to the immense size of the data, he didn't manage to even load the data to the tool. You are planning to 1) load the datasets from 10 different branches to Hadoop Distributed File System (HDFS), 2) preprocess them using HDFS, and 3) use MapReduce programming model to discover a comprehensive process model. 

### Preparation
The preparation of this problem consists of two steps:

**Preparation step 1**: Replace the filepath to your own filepath to produce the **LoanApplication.csv**.  

In [ ]:
import pandas as pd
#your filepath
filepath = "./dataset/LoanApplication.csv"
original_log = pd.read_csv(filepath,sep=",")
original_log.head()

**Preparation step 2**: In this question, we generate 10 event logs based on the ``original_log``. For randomization, you need to use the sum of the group's matriculation numbers (e.g., a group with 3 students having "100000", "100001", and "100002" as their matriculation numbers will use "300003" for the randomization).

In [ ]:
# Below are GIVEN utility functions (do not modify):
import random
import os
def _ramdomize(x):
    random_val = random.randint(5,10)
    return x+random_val

def _randomize_log(log,matriculation_num):
    """Randomize case attributes based on the matriculation number

    Keyword arguments:
    log -- event log
    matriculation_num - sum of matriculation numbers
    """
    attribute_cols = ["Duration"]
    random.seed(matriculation_num)
    for attr in attribute_cols:
        log[attr] = log[attr].apply(_ramdomize)
    return log

def _extract_log(log,iter_num):
    """Extract n-th log to ./generated_logs/

    Keyword arguments:
    log -- event log
    iter_num -- n-th iteration
    """
    log.to_csv("./generated_logs/generated_log-{}.tsv".format(iter_num),header=False,index=False, sep="\t",line_terminator="")

def generate_log(original_log,num_replication,mat_num):
    """Generate logs (randomized by the matriculation number and extracted to ./generated_logs/) 

    Keyword arguments:
    log -- event log
    num_replication -- number of generated logs
    mat_num -- sum of matriculation numbers
    """
    case_col="CaseID"
    timestamp_col = "Timestamp"
    dir_path = "./generated_logs"
    try:
        os.mkdir(dir_path)
    except OSError:
        print ("Directory already exists: %s" % dir_path)
    else:
        print ("Successfully created the directory %s " % dir_path)
    
    for i in range(num_replication):
        print("starts {}".format(i))
        generated_log = original_log.copy(deep=True)
        generated_log[case_col] = str(i) + generated_log[case_col]
        generated_log[timestamp_col] = generated_log[timestamp_col].apply(str).str.zfill(10)
        randomized_log = _randomize_log(generated_log,mat_num)
        _extract_log(randomized_log,i)
        print ("Successfully created %i th log at %s "% (i,dir_path))

Replace the SUM_MAT_NUM to yours to generate logs.

In [ ]:
# your code
SUM_MAT_NUM = 154031 
NUM_REPITITION=10
generate_log(original_log,NUM_REPITITION,SUM_MAT_NUM)

### 6.1. Hadoop Distributed File System (HDFS)

Now, it's time to work with the Hadoop Distributed File System (HDFS). The goal of this task is to merge 10 event logs at your disk using HDFS. Follow the instructions below and show your results in each step (screenshots of the command line). We use "letter identifier" for this task (The letter identifier is the string consisting of the first letters of the group memebers' first names, e.g., for the group with "Antonio Rüdiger", "Bernd Leno", "Christian Günter", the indentifier is "ABC").

    1) Import the event logs to your Docker engine (at /usr/local/hadoop/(identifier)-generated-logs/).
    2) Upload the event logs to the running HDFS (at /input/(identifier)-generated-logs/). 
    3) Merge all the files and copy the result back to HDFS (at /input/(identifier)-final-log-10.tsv).
    4) Merge 6 files (you can randomly select) and copy the result back to HDFS (at /input/(identifier)-final-log-6.tsv).
    5) Merge 2 files (you can randomly select) and copy the result back to HDFS (at /input/(identifier)-final-log-2.tsv).
    6) Print out the completely-merged event log from 3), i.e., "(identifier)-final-log-10.tsv", in the command line (the screenshot may contain 10 rows).

In [ ]:
#your code
from IPython.display import Image
# Image(filename='your_path_to_screenshot_of_a1')

In [ ]:
#your code
#Image(filename='your_path_to_screenshot_of_a2') 

In [ ]:
#your code
#Image(filename='your_path_to_screenshot_of_a3') 

In [ ]:
#your code
#Image(filename='your_path_to_screenshot_of_a4') 

### 6.2. Process Discovery

Discover a process model from the completely merged event log using MapReduce algorithms. Explain how you discover the process model with the following deliverables:

    1) Mapper function (as python file(s))
    2) Reducer function (as python file(s))
    3) Hadoop commands for MapReduce computation (as text file)
    4) Jupyter notebook script that visualize (1) a directly-follows graph and (2) a Petri net  based on the computed directly-follows relations.

<font color='red'>Important!</font> Please note that in this task, your result will be evaluated based on whether they are reproducible from your explanation. If you skip MapReduce computations for this task, you will get 0 points.The deliverables of 1), 2), and 3) should be submitted as outputs. 

In [ ]:
# your code

### 6.3. Performance Analysis

a) Compute the total service time for each case based on MapReduce algorithms using the completely-merged event log (i.e., (identifier)-final-log-10.tsv) and visualize 100 cases that show the longest total service time using any chart.
    
The deliverables of 1), 2), 3) and 4) should be submitted as outputs:
```
1) Mapper function (as python file(s))
2) Reducer function (as python file(s))
3) Hadoop commands for MapReduce calculation (as text file)
4) Result: total service times for cases (as text file)
```

b) Compare the (approximate) computation time of the service time calculation between 1. the completely-merged event log (i.e., (identifier)-final-log-10.tsv), 2. 6-merged event log (i.e., (identifier)-final-log-6.tsv), and 3. 2-merged event log (i.e., (identifier)-final-log-2.tsv). Interpret the difference (e.g., the computation time scales linearly with the increasing number of events).

In [ ]:
# your code